# Generating Magic Numbers for EVM

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

def gen_powers(base) -> list:
    """
    Generates a list of powers of a number.
    """
    powers = []
    for q in [10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001]:
        place = [round((base ** power), ndigits=10) for power in np.linspace(0, q, 11)][1:-1]
        powers.append(place)
    return powers

def power_contract(base, pragma, title, description, license, author, input, output) -> str:
    """
    Generates a Vyper contract that calculates the power of a number.
    """
    preamble = \
    """#@version {pragma}
\"""
@title {title}
@description {description}
@license {license}
@author {author}
@input {input}
@output {output}
\"""
"""

    preamble = preamble.format(pragma=pragma, title=title, description=description, \
        license=license, author=author, input=input, output=output)
    
    magic_numbers = str(gen_powers(base))

    body = \
"""
_owner: private(address)
tab: constant(decimal[10][10]) = {magic_numbers}

@external
def __init__():
    self._owner = msg.sender

@pure
def multiply_exponents(n: decimal) -> decimal:
    # this function refers to the table and returns x^n 
    # by multiplying values in 10 decimal places
    # param: n = the power to raise e to
    
    # split n into a whole part and a decimal part
    r: int256 = floor(n)
    f: decimal = n - convert(r, decimal)

    # create a list of every digit in f
    digits: int128[10]
    for i in range(9):
        digits[i] = floor(f * convert((10 ** i), decimal))
    
    # calculate the two parts and multiply them together
    whole: decimal = convert(r // 9, decimal) * tab([0][9]) * tab([0][r % 9])
    fraction: decimal = 1.0
    for i in range(9):
        fraction *= tab(i+1, digits[i])
    return whole * fraction

@external
def exp():
    # calls the multiply_exponents function

    assert msg.data >= 0.0, "Negative powers are not supported."
    send(msg.sender, multiply_exponents(msg.data))

@external
def unalive():
    # allows the owner to unalive the contract

    assert msg.sender == self._owner, "Only the owner can unalive the contract."
    selfdestruct(self._owner)

@external
def change_owner(new_owner: address):
    # changes the owner of the contract
    # param: new_owner = the new owner of the contract

    assert msg.sender == self._owner, "Only the owner can change the owner."
    self.owner = new_owner
"""
    txt = preamble + body.format(magic_numbers=magic_numbers)
    with open(title + '.vy', 'w') as f:
        f.write(txt)



#### Powers of e

In [19]:
power_contract(base=np.exp(1),
               pragma="0.3.7",
               title="e",
               description="base of the natural logarithm.",
                license="MIT",
                author="magic numbers",
                input="a non-negative decimal power of e",
                output="a^n, a decimal number")

#### Powers of pi

In [20]:
power_contract(base = np.pi,
               pragma="0.3.7", title="pi",
                description="pi is the signature of a circle.", license="MIT",
                author="magic numbers", input="n, a decimal power of pi",
                output="pi^n, a decimal number")

#### Powers of tau

In [21]:
power_contract(base=np.pi*2,
                pragma="0.3.7",
                title="tau",
                description="tau is also the signature of a wave.",
                license="MIT",
                author="Magic Numbers",
                input="n, a decimal power of tau",
                output="tau^n, a decimal number")